# Gemetric Test of Reciprocity Moment Tensors

### Step 0

Load packages

In [ ]:
#load all packages
import datetime
import pickle
import copy
import os

from sys import argv
from pathlib import Path

import numpy as np
import pandas as pd
import pyvista as pv
import matplotlib.pyplot as plt 
from matplotlib.colors import Normalize


from pyaspect.project import *
from pyaspect.model.gridmod3d import gridmod3d as gm
from pyaspect.model.bbox import bbox as bb
from pyaspect.model.gm3d_utils import *
from pyaspect.moment_tensor import MomentTensor
from pyaspect.specfemio.headers import *
from pyaspect.specfemio.write import *
from pyaspect.specfemio.write import _write_header
from pyaspect.specfemio.read import *
from pyaspect.specfemio.utils import *


import pyaspect.events.gevents as gevents
import pyaspect.events.gstations as gstations
from pyaspect.events.munge.knmi import correct_station_depths as csd_f
import pyaspect.events.mtensors as mtensors
from obspy.imaging.beachball import beach
from obspy import UTCDateTime
import shapefile as sf

from pyrocko.moment_tensor import MomentTensor as RockoMT

### Step 1 

Extract the ndarray of the subsampled, smoothed NAM model and instantiate a new GriddedModel3D object for QC'ing

In [ ]:
data_in_dir  = 'data/output/'
data_out_dir = data_in_dir
!ls {data_in_dir}
!ls data/groningen

### Step 6 

Decompress the ndarray of the sliced, subsampled, smoothed NAM model and instantiate a new GriddedModel3D object for QC'ing

In [ ]:
# set filename then used it to decompress model
ifqn = f'{data_out_dir}/vsliced_subsmp_smth_nam_2017_vp_vs_rho_Q_model_dx100_dy100_dz100_maxdepth5850_sig250.npz'
vslice_gm3d, other_pars = decompress_gm3d_from_file(ifqn)

print()
print('decompressed gridded model\n:',vslice_gm3d) 
print()
print('other parameters:\n',other_pars)
print()

# WARNING: this will unpack all other_pars, if you overwrite a variable of the samename as val(key), then you 
#          may not notice, and this may cause large headaches.  I use it because I am aware of it.
'''
for key in other_pars:
    locals()[key] = other_pars[key]  #this is more advanced python than I think is reasonable for most 
sig_meters = sig
''';

# another way to get these varibles is just use the accessor functions for the gridmod3d.  We need them later.
xmin = other_pars['xmin']
dx   = other_pars['dx']
nx   = other_pars['nx']
ymin = other_pars['ymin']
dy   = other_pars['dy']
ny   = other_pars['ny']
zmin = other_pars['zmin']
dz   = other_pars['dz']
nz   = other_pars['nz']
sig_meters = other_pars['sig']  # this variable is used later
print('sig_meters:',sig_meters)

In [ ]:
# Create the spatial reference
grid = pv.UniformGrid()

# Set the grid dimensions: shape + 1 because we want to inject our values on
#   the CELL data
nam_dims = list(vslice_gm3d.get_npoints())
nam_origin = [0,0,-vslice_gm3d.get_gorigin()[2]]
#nam_origin = list(vslice_gm3d.get_gorigin())
#nam_origin[2] *= -1
nam_origin = tuple(nam_origin)
nam_spacing = list(vslice_gm3d.get_deltas())
nam_spacing[2] *=-1
nam_spacing = tuple(nam_spacing)
print('nam_dims:',nam_dims)
print('nam_origin:',nam_origin)
print('nam_spacing:',nam_spacing)

# Edit the spatial reference
grid.dimensions = np.array(nam_dims) + 1
grid.origin = nam_origin  # The bottom left corner of the data set
grid.spacing = nam_spacing  # These are the cell sizes along each axis
nam_pvalues = vslice_gm3d.getNPArray()[0]
print('pvalues.shape:',nam_pvalues.shape)

# Add the data values to the cell data
grid.cell_arrays["values"] = nam_pvalues.flatten(order="F")  # Flatten the array!

# Now plot the grid!
cmap = plt.cm.jet
#grid.plot(show_edges=True,cmap=cmap)
grid.plot(cmap=cmap,opacity=1.0)


In [ ]:
slices = grid.slice_orthogonal()

#slices.plot(show_edges=True,cmap=cmap)
slices.plot(cmap=cmap)

## create virtual recievers (CMT solutions in forwards sense)

In [ ]:
#coords = vslice_gm3d.getGlobalCoordsPointsXYZ()
coords = vslice_gm3d.getLocalCoordsPointsXYZ()
coords[:,2] = -coords[:,2]

xc = np.unique(coords.T[0,:])
yc = np.unique(coords.T[1,:])
zc = np.unique(coords.T[2,:])


#n_rand_p = 1000

n_rand_p = 3
np.random.seed(n_rand_p) #nothing special about using n_rand_p just want reproducible random

#stay away from the edges of the model for derivatives 
# and to avoid boundary effects
xy_pad = 500 

lrx = np.min(xc) + xy_pad
lry = np.min(yc) + xy_pad
lrz = -3400.0

hrx = np.max(xc) - xy_pad
hry = np.max(yc) - xy_pad
hrz = -2600.0

srx = hrx - lrx
sry = hry - lry
srz = hrz - lrz

    
vrec_cmt_xyz = np.array([lrx + 0.33*srx,lry + 0.33*sry,-3000],dtype=np.float32).reshape((1,3))
    

print('cmt_xyz:\n',vrec_cmt_xyz)


In [ ]:
pv_rpoints = pv.wrap(vrec_cmt_xyz)
p = pv.Plotter()
slices = grid.slice_orthogonal()
#p.add_mesh(slices,cmap=cmap,opacity=0.50)
#p.add_mesh(slices,cmap=cmap,opacity=1)
p.add_mesh(grid,cmap=cmap,opacity=0.50)
p.add_mesh(pv_rpoints, render_points_as_spheres=True, point_size=5,opacity=1.0)

p.show()

## Make Moment Tensors and CMTSolutionHeaders for each tensor

In [ ]:
def CMTtoM0(CMTsol):
    A = np.array(([CMTsol[0],CMTsol[3],CMTsol[4]],
                  [CMTsol[3],CMTsol[1],CMTsol[5]],
                  [CMTsol[4],CMTsol[5],CMTsol[2]]))
    M0 = ((1/np.sqrt(2))*np.sqrt(np.sum(A*A)))
    
    return(M0)

def aki_from_sdr(strike,dip,rake,M0):
    from math import sin,cos
    
    print('input M0:',M0)
    """
    converts given strike/dip/rake to moment tensor
    """
    S = strike
    D = dip
    R = rake

    # PI / 180 to convert degrees to radians
    d2r =  0.017453293

    print("Strike    = %9.5f degrees" % S)
    print("Dip       = %9.5f degrees" % D)
    print("Rake/Slip = %9.5f degrees" % R)
    print("")

    # convert to radians
    S *= d2r
    D *= d2r
    R *= d2r

    '''
    # Aki & Richards
    Mxx = -1.0 * ( sin(D) * cos(R) * sin (2*S) + sin(2*D) * sin(R) * sin(S)*sin(S) )
    Myy =        ( sin(D) * cos(R) * sin (2*S) - sin(2*D) * sin(R) * cos(S)*cos(S) )
    Mzz = -1.0 * ( Mxx + Myy)
    Mxy =        ( sin(D) * cos(R) * cos (2*S) + 0.5 * sin(2*D) * sin(R) * sin(2*S) )
    Mxz = -1.0 * ( cos(D) * cos(R) * cos (S)   + cos(2*D) * sin(R) * sin(S) )
    Myz = -1.0 * ( cos(D) * cos(R) * sin (S)   - cos(2*D) * sin(R) * cos(S) )
    ''';
    
            #Aki and Richards
    Mxx = -( np.sin(D)*np.cos(R)*np.sin(2*S) + np.sin(2*D)*np.sin(R)*(np.sin(S)**2) )
    Myy =  ( np.sin(D)*np.cos(R)*np.sin(2*S) - np.sin(2*D)*np.sin(R)*(np.cos(S)**2) )
    Mzz = -( Mxx + Myy )
    Mxy =  ( np.sin(D)*np.cos(R)*np.cos(2*S) + 0.5*np.sin(2*D)*np.sin(R)*np.sin(2*S) )
    Mxz = -( np.cos(D)*np.cos(R)*np.cos(S)   + np.cos(2*D)*np.sin(R)*np.sin(S) )
    Myz = -( np.cos(D)*np.cos(R)*np.sin(S)   - np.cos(2*D)*np.sin(R)*np.cos(S) )
    

    a_mt = np.array([Mxx,Myy,Mzz,Mxy,Mxz,Myz])
    a_mt *= M0
    
    # Harvard CMT
    Mtt = a_mt[0] #Mxx 
    Mpp = a_mt[1] #Myy 
    Mrr = a_mt[2] #Mzz 
    Mtp = -1.0*a_mt[3] #Mxy
    Mrt = a_mt[4] #Mxz 
    Mrp = -1.0*a_mt[5] #Myz
    
    h_mt = np.array([Mrr,Mtt,Mpp,Mrt,Mrp,Mtp])
    
    

    print("Aki&Richards1980:  Mxx  Myy  Mzz  Mxy  Mxz  Myz")
    print("%9.5f %9.5f %9.5f %9.5f %9.5f %9.5f\n" %(tuple(a_mt)))
    print("M0:",CMTtoM0(a_mt))
    print()
    print("Harvard:  Mrr  Mtt  Mpp  Mrt  Mrp  Mtp")
    print("%9.5f %9.5f %9.5f %9.5f %9.5f %9.5f\n" %(tuple(h_mt)))
    print("M0:",CMTtoM0(h_mt))
    print()
    
    return a_mt

In [ ]:
# this is the path to the project dir on the cluster
my_proj_dir = '/scratch/seismology/tcullison/test_mesh/FWD_Batch_Src_Test'

m0 = 1
mW = (np.log10(m0)-9.1)/1.5


#(mnn, mee, mdd, mne, mnd, med, magnitude)
#((mnn, mne, mnd), (mne, mee, med), (mnd, med, mdd))
h_mat_xy = np.array([[0,0,0],[0,0,-1],[0,-1,0]])
h_mat_xz = np.array([[0,0,1],[0,0,0],[1,0,0]])
h_mat_yz = np.array([[0,-1,0],[-1,0,0],[0,0,0]])
Mxy = MomentTensor(m_up_south_east=h_mat_xy)  #[[0,1,0],[1,0,0],[0,0,0]]  SPEC coord system
Mxz = MomentTensor(m_up_south_east=h_mat_xz)  #[[0,0,0],[0,0,1],[0,1,0]]  SPEC coord system
Myz = MomentTensor(m_up_south_east=h_mat_yz)  #[[0,0,1],[0,0,0],[1,0,0]]  SPEC coord system
print(f'Mxy: {Mxy}')
print(f'Mxy PyR:       {Mxy.m6()}')
print(f'Mxy Aki:       {Mxy.m6_east_north_up()}')
print(f'Mxy Har:       {Mxy.m6_up_south_east()}\n\n')
print(f'Mxz: {Mxz}')
print(f'Mxz PyR:       {Mxz.m6()}')
print(f'Mxz Aki:       {Mxz.m6_east_north_up()}')
print(f'Mxz Har:       {Mxz.m6_up_south_east()}\n\n')
print(f'Myz: {Myz}')
print(f'Myz PyR:       {Myz.m6()}')
print(f'Myz Aki:       {Myz.m6_east_north_up()}')
print(f'Myz Har:       {Myz.m6_up_south_east()}\n\n')

l_mt = [('Harvard-XY',Mxy),('Harvard-XZ',Mxz),('Harvard-YZ',Myz)]

for mt in l_mt:
    print(f'mt: {mt}')
    
l_cmt_srcs = []
for i in range(len(l_mt)):
    cmt_h = CMTSolutionHeader(date=datetime.datetime.now(),
                              ename=l_mt[i][0],
                              #ename=f'Event-{str(i).zfill(4)}',
                              tshift=0.0,
                              hdur=0.0,
                              lat_yc=vrec_cmt_xyz[0,1],
                              lon_xc=vrec_cmt_xyz[0,0],
                              depth=-vrec_cmt_xyz[0,2],
                              mt=l_mt[i][1],
                              #mt=l_mt[i],
                              eid=i,
                              sid=0)
    l_cmt_srcs.append(cmt_h)
    
print()
for cmt in l_cmt_srcs:
    print(f'cmt:\n{cmt}')
    
#assert False

## Make Corresponding "Virtual" Recievers (including cross membors for derivatives) for the CMT's

In [ ]:
m_delta = 50.0 # distance between cross stations for derivatives
assert m_delta < xy_pad #see cells above this is padding
#l_grp_vrecs = make_grouped_half_cross_reciprocal_station_headers_from_cmt_list(l_cmt_srcs,m_delta)
l_grp_vrecs = make_grouped_cross_reciprocal_station_headers_from_cmt_list(l_cmt_srcs,m_delta)

ig = 0
for grp in l_grp_vrecs:
    print(f'***** Group: {ig} *****\n')
    ir = 0
    for gvrec in grp:
        print(f'*** vrec: {ir} ***\n{gvrec}')
        ir += 1
    ig += 1

print(len(flatten_grouped_headers(l_grp_vrecs)))
    

## Plot Virtual Receiver Groups

In [ ]:
all_g_xyz = get_xyz_coords_from_station_list(flatten_grouped_headers(l_grp_vrecs))
all_g_xyz[:,2] *= -1 #pyview z-up positive and oposize sign of standard geophysics 
pv_all_points = pv.wrap(all_g_xyz)
p = pv.Plotter()
p.add_mesh(grid,cmap=cmap,opacity=0.5)
#p.add_mesh(slices,cmap=cmap,opacity=1.0)
p.add_mesh(pv_all_points, render_points_as_spheres=True, point_size=5,opacity=1.0)
p.show()

## Make real-receivers/virtual-sources

In [ ]:
h = 3000
rec_z = -200
vsrc_rec_xyz = np.zeros((9,3))

for i in range(vsrc_rec_xyz.shape[0]):
    vsrc_rec_xyz[i,:] = vrec_cmt_xyz[0,:]
    vsrc_rec_xyz[i,2] = rec_z
    
# x-h, y-y
vsrc_rec_xyz[0,0] = vrec_cmt_xyz[0,0] - h   
vsrc_rec_xyz[0,1] = vrec_cmt_xyz[0,1] - h   

# x, y-y
vsrc_rec_xyz[1,1] = vrec_cmt_xyz[0,1] - h   
    
# x+h, y-y
vsrc_rec_xyz[2,0] = vrec_cmt_xyz[0,0] + h   
vsrc_rec_xyz[2,1] = vrec_cmt_xyz[0,1] - h   
    
# x-h, y
vsrc_rec_xyz[3,0] = vrec_cmt_xyz[0,0] - h   

# x, y
#do nothing but skip to next index below

# x+h, y
vsrc_rec_xyz[5,0] = vrec_cmt_xyz[0,0] + h   

# x-h, y+y
vsrc_rec_xyz[6,0] = vrec_cmt_xyz[0,0] - h   
vsrc_rec_xyz[6,1] = vrec_cmt_xyz[0,1] + h   

# x, y+y
vsrc_rec_xyz[7,1] = vrec_cmt_xyz[0,1] + h   

# x+h, y+y
vsrc_rec_xyz[8,0] = vrec_cmt_xyz[0,0] + h   
vsrc_rec_xyz[8,1] = vrec_cmt_xyz[0,1] + h   



## Plot virtual sources (red) with virtual receivers (white)

In [ ]:
pv_spoints = pv.wrap(vsrc_rec_xyz)
p = pv.Plotter()
#p.add_mesh(slices,cmap=cmap,opacity=0.50)
p.add_mesh(grid,cmap=cmap,opacity=0.3)
p.add_mesh(pv_spoints, render_points_as_spheres=True, point_size=8,opacity=1,color='red')
#p.add_mesh(pv_rpoints, render_points_as_spheres=True, point_size=5,opacity=0.5)
p.add_mesh(all_g_xyz, render_points_as_spheres=True, point_size=5,opacity=0.5)
p.show()

## Make StationHeaders (real recievers/virtual sources) 

In [ ]:
l_real_recs = []
for i in range(len(vsrc_rec_xyz)):
    
    tr_bname = 'tr'
    new_r = StationHeader(name=tr_bname,
                          network='NL', #FIXME
                          lon_xc=vsrc_rec_xyz[i,0],
                          lat_yc=vsrc_rec_xyz[i,1],
                          depth=-vsrc_rec_xyz[i,2], #specfem z-down is positive
                          elevation=0.0,
                          trid=i)
    l_real_recs.append(new_r)
    
for rec in l_real_recs:
    print(rec)


## Make ForceSolutionHeaders for the above virtual sources (including force-triplets for calculation derivatives)

In [ ]:
l_grp_vsrcs = make_grouped_reciprocal_force_solution_triplet_headers_from_rec_list(l_real_recs)

## Make replicates of each virtual receiver list: one for each force-triplet

In [ ]:
l_grp_vrecs_by_vsrcs = make_replicated_reciprocal_station_headers_from_src_triplet_list(l_grp_vsrcs,
                                                                                          l_grp_vrecs)

## Plot virtual sources (red) and virtual receivers (white) FROM headers

In [ ]:
grp_s_xyz = get_unique_xyz_coords_from_solution_list(flatten_grouped_headers(l_grp_vsrcs))
grp_s_xyz[:,2] *= -1 #pyvista z-up is positive

flat_recs = flatten_grouped_headers(flatten_grouped_headers(l_grp_vrecs_by_vsrcs))
grp_r_xyz = get_unique_xyz_coords_from_station_list(flat_recs)
grp_r_xyz[:,2] *= -1 #pyvista z-up is positive

print(len(grp_s_xyz))
print(len(grp_r_xyz))

pv_spoints = pv.wrap(grp_s_xyz)
pv_rpoints = pv.wrap(grp_r_xyz)

p = pv.Plotter()
p.add_mesh(slices,cmap=cmap,opacity=0.50)
p.add_mesh(grid,cmap=cmap,opacity=0.3)
p.add_mesh(pv_spoints, render_points_as_spheres=True, point_size=8,opacity=1,color='red')
p.add_mesh(pv_rpoints, render_points_as_spheres=True, point_size=5,opacity=0.5)
p.show()

## Make replicates of each "real" receiver list: for each CMT source

In [ ]:
l_grp_recs_by_srcs = make_replicated_station_headers_from_src_list(l_cmt_srcs,l_real_recs)


for i in range(len(l_cmt_srcs)):
    print(f'***** SRC Records for Source: {i} *****\n')
    for j in range(len(l_real_recs)):
        print(f'*** REC Header for Receiver: {j} ***\n{l_grp_recs_by_srcs[i][j]}')
    

## Plot "real" sources (red) and virtual receivers (white) FROM headers

In [ ]:
grp_s_xyz = get_unique_xyz_coords_from_solution_list(l_cmt_srcs)
grp_s_xyz[:,2] *= -1 #pyvista z-up is positive

flat_recs = flatten_grouped_headers(l_grp_recs_by_srcs) #real!
grp_r_xyz = get_unique_xyz_coords_from_station_list(flat_recs)
grp_r_xyz[:,2] *= -1 #pyvista z-up is positive

print(len(grp_s_xyz))
print(len(grp_r_xyz))

pv_spoints = pv.wrap(grp_s_xyz)
pv_rpoints = pv.wrap(grp_r_xyz)

p = pv.Plotter()
p.add_mesh(slices,cmap=cmap,opacity=0.50)
p.add_mesh(grid,cmap=cmap,opacity=0.3)
p.add_mesh(pv_spoints, render_points_as_spheres=True, point_size=12,opacity=1,color='red')
p.add_mesh(pv_rpoints, render_points_as_spheres=True, point_size=8,opacity=0.5)
p.show()

#assert False

## Make reciprical RecordHeader

In [ ]:
l_flat_vsrcs = flatten_grouped_headers(l_grp_vsrcs)
l_flat_vrecs = flatten_grouped_headers(flatten_grouped_headers(l_grp_vrecs_by_vsrcs))

vrecord_h = RecordHeader(name='Reciprocal-Record',solutions_h=l_flat_vsrcs,stations_h=l_flat_vrecs)

# save the header to disc
vrec_fqp = os.path.join(data_out_dir,'simple_record_h')
_write_header(vrec_fqp,vrecord_h)

#verify file is there
!ls -l {vrec_fqp}

## Make reciprocal project

In [ ]:
test_proj_name = 'ReciprocalGeometricTestProject'
test_proj_root_fqp =  os.path.join(data_out_dir, 'tmp/TestProjects/NewMKProj')
test_parfile_fqp =  os.path.join(data_out_dir, 'Par_file')
test_mesh_fqp = '/scratch/seismology/tcullison/test_mesh/MESH-default_batch_force_src'
test_spec_fqp = '/quanta1/home/tcullison/DevGPU_specfem3d'
test_pyutils_fqp = '/quanta1/home/tcullison/myscripts/python/specfem/pyutils'
test_script_fqp = '/quanta1/home/tcullison/myscripts/specfem'

#copy the reciprocal record
test_proj_record_h = vrecord_h.copy()

make_fwd_project_dir(test_proj_name,
                     test_proj_root_fqp,
                     test_parfile_fqp,
                     test_mesh_fqp,
                     test_spec_fqp,
                     test_pyutils_fqp,
                     test_script_fqp,
                     test_proj_record_h,
                     copy_mesh=False,
                     batch_srcs=False,
                     verbose=True,
                     max_event_rdirs=MAX_SPEC_SRC)
                     #max_event_rdirs=)
        

print()
print('ls:')
!ls {test_proj_root_fqp}
print('ls:')
!ls {test_proj_root_fqp}/*/*


## Make Forward/Real RecordHeader

In [ ]:
l_flat_srcs = l_cmt_srcs #NOTE: we don't need to flatten CMT list because they are not grouped
l_flat_recs = flatten_grouped_headers(l_grp_recs_by_srcs) #Note: only one level of flattening

record_h = RecordHeader(name='Forward-Record',solutions_h=l_flat_srcs,stations_h=l_flat_recs)
print(f'Forward Record:\n{record_h}')

# save the header to disc
rec_fqp = os.path.join(data_out_dir,'real_simple_record_h')
_write_header(rec_fqp,record_h)

#verify file is there
!ls -l {rec_fqp}

print('l_flat_srcs:',type(l_flat_srcs[0]))

assert False

## Make "real" project

In [ ]:
test_real_proj_name = 'ForwardGeometricTestProject'
test_proj_root_fqp =  os.path.join(data_out_dir, 'tmp/TestProjects/NewMKProj')
test_parfile_fqp =  os.path.join(data_out_dir, 'Par_file')
test_mesh_fqp = '/scratch/seismology/tcullison/test_mesh/MESH-default_batch_force_src'
test_spec_fqp = '/quanta1/home/tcullison/DevGPU_specfem3d'
test_pyutils_fqp = '/quanta1/home/tcullison/myscripts/python/specfem/pyutils'
test_script_fqp = '/quanta1/home/tcullison/myscripts/specfem'

#copy the forward/real record
test_real_proj_record_h = record_h.copy()

make_fwd_project_dir(test_real_proj_name,
                     test_proj_root_fqp,
                     test_parfile_fqp,
                     test_mesh_fqp,
                     test_spec_fqp,
                     test_pyutils_fqp,
                     test_script_fqp,
                     test_real_proj_record_h,
                     copy_mesh=False,
                     batch_srcs=False,
                     verbose=True,
                     max_event_rdirs=MAX_SPEC_SRC)
                     #max_event_rdirs=2)


print()
print('ls:')
!ls {test_proj_root_fqp}
print('ls:')
!ls {test_proj_root_fqp}/*/*